In [1]:
from pathlib import Path

import geopandas as gpd
import folium
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import random
import requests
import yaml
from bmi_topography import Topography
from ipywidgets import interact
from matplotlib.colors import LogNorm
from shapely.geometry import Polygon, shape
from skimage.measure import find_contours, label
from tqdm.notebook import tqdm

In [2]:
CONFIG_FILE = "../config.yml"
with open(CONFIG_FILE) as handle:
    config = yaml.safe_load(handle)


data_dir = Path(config['global']['data_dir'])
slr_path = data_dir / config['demo']['slr_file']
damage_path = data_dir / config['demo']['damage_file']
topo_cache_dir = data_dir / config['demo']['topo_cache_subdir']

opentopo_api_key = config['demo']['opentopo_api_key']

In [3]:
slr_data = pd.read_csv(slr_path).pivot(index='City', columns='Year', values='SLR')
# display(slr_data)
damage_data = pd.read_csv(damage_path, index_col='City')
damage_data.columns = slr_data.columns
# display(damage_data)
# print(damage_data.index.symmetric_difference(slr_data.index))

In [4]:
@interact(city=slr_data.index, year=slr_data.columns)
def plot(city, year):
    fig, ax = plt.subplots(figsize=(12, 6))

    slr_data.loc[city].plot(label="Sea level rise (left)", ax=ax)
    ax.scatter(year, slr_data.loc[city, year], c='r', s=100, zorder=2)
    ax.set_ylabel("Mean modelled SLR (metres)")
    ax.set_title(city)

    ax2 = damage_data.loc[city].plot(secondary_y=True, ax=ax, label="Damages")
    ax2.scatter(year, damage_data.loc[city, year], c='r', s=100, zorder=2)
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc=0)
    ax2.set_ylabel("Expected damages under RCP8.5 (million USD (2005))")
    
    
    url = f"https://nominatim.openstreetmap.org/search?q={city}&polygon_geojson=1&format=json"
    results = requests.get(url, headers={'User-Agent': 'test'})
    results.raise_for_status()
    top = next(r for r in results.json() if r['class'] == 'boundary' and r['type'] == 'administrative')
    
    location = (float(top['lat']), float(top['lon']))
    bbox = [float(s) for s in top['boundingbox']]
    
    f = folium.Figure(width=12*65, height=6*65)
    m = folium.Map(location, tiles='CartoDB positron')
    m.add_to(f)

    boundary = gpd.GeoDataFrame(geometry=[shape(top['geojson'])], crs='epsg:4326')
    folium.GeoJson(
        boundary,
        style_function=lambda _: {'weight': 0}
    ).add_to(m)

    topo = Topography(
        dem_type='COP90',
        south=bbox[0],
        north=bbox[1],
        west=bbox[2],
        east=bbox[3],
        output_format='GTiff',
        cache_dir=topo_cache_dir,
        api_key=opentopo_api_key
    )
    print("Downloading data...", end=" ")
    topo.fetch()
    print("done.")
    data = topo.load().sel(band=1)
    # display(data)
    # fig, ax = plt.subplots(figsize=(12, 6))
    # data.plot(cmap='Greys', vmin=data.min(), vmax=20, ax=ax)
    # print(slr_data.loc[city, year])
    flood = (data >= 1e-6) & (data <= slr_data.loc[city, year])
    labels, num_labels = label(flood, return_num=True)
    # ax.imshow(flood, cmap='tab10')
    if num_labels:
        sx, _, tx, _, sy, ty = data.attrs['transform']
        polygons = []
        print("Computing levels...")
        for i in tqdm(range(1, num_labels)):
            label_map = labels == i
            if label_map.sum() < 4:
                continue
            # Add 1px wide zero-padding to close all contours
            label_map = np.pad(label_map, 1)
            # Substract 1 to remove padding, then scale and translate to
            # put in map coordinates.
            contours = [
                np.fliplr(contour - 1) * [sx, sy] + [tx, ty]
                for contour in find_contours(label_map, 0)
            ]
            # The exterior bound has the largest area.
            areas = np.array([Polygon(c).area for c in contours])
            exterior_id = np.argmax(areas)
            # print(areas[exterior_id])
            if areas[exterior_id] < 1e-6:
                continue
            # print("Ho")
            exterior = contours.pop(exterior_id)
            polygons.append(Polygon(exterior, contours).buffer(0))
            # ax.plot(contour[:, 0], contour[:, 1], linewidth=2)
        # data[0].plot.contourf(levels=[0, 10], ax=ax, colors=['none', (1, 0, 0, 0.2), 'none'])
        # print([p.centroid.coords[0] for p in polygons])

        if polygons:
            polygons_gdf = gpd.GeoDataFrame(
                geometry=polygons,
                crs=data.attrs['crs'].split('=')[1]
            ).sjoin(boundary)
            if not polygons_gdf.empty:
                folium.GeoJson(
                    polygons_gdf,
                    style_function=lambda _: {'color': 'red'}  #random.choice(['red', 'black', 'blue'])}
                ).add_to(m)
    
    display(f)


interactive(children=(Dropdown(description='city', options=('Abidjan', 'Accra', 'Adelaide', 'Alexandria', 'Alg…